In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Cristovao Beirao da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

In [1]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from fnmatch import fnmatch
import scipy.stats as stats
import hist
import mplhep as hep
plt.style.use(hep.style.CMS)
from notebooks.notebook_helpers import *

from scripts.log_action import log_action_v2
from time import sleep

In [2]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2_EPIR_Pair11"]
# chip_names = ["ET2-W36-IP5-20", "ET2-W36-IP7-13", "ET2-W36-IP7-10"]
# old 
# chip_names = ["ET2-W36-IP7-12", "ET2-W36-IP5-15", "ET2-W36-IP5-14"]
# old [0x78, 0x61, 0x74]
# old top modified C type =  ET2-W36-IP5-15
# replaced with new type D = ET2-W36-IP7-10
chip_fignames = chip_names
chip_figtitles = ["ET2_BumpBond_EPIR_Pair11"]

# 'The port name the USB-ISS module is connected to. Default: COM3'
port = "/dev/ttyACM2"
# I2C addresses for the pixel block and WS
chip_addresses = [0x60]
# chip_addresses = [0x78, 0x74, 0x61]
ws_addresses = [None, None, None]

today = datetime.date.today().isoformat()
# today = "2023-09-14"


# i2c_gui.__no_connect__ = False  # Set to fake connecting to an ETROC2 device
# i2c_gui.__no_connect_type__ = "echo"  # for actually testing readback
# #i2c_gui.__no_connect_type__ = "check"  # default behaviour

hostname = "192.168.2.3"

polarity = "0x000f"

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (today + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

In [3]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,ws_addresses,chip_names,[("1","1")])
# i2c_conn = i2c_connection(port,chip_addresses,chip_names,[("1","1"), ("1","1"), ("1","1")])

In [4]:
chip = i2c_conn.get_chip_i2c_connection(0x60)

row_indexer_handle,_,_ = chip.get_indexer("row")
column_indexer_handle,_,_ = chip.get_indexer("column")
broadcast_handle,_,_ = chip.get_indexer("broadcast")

## Similar to Xing

In [5]:
flipped_values = [None for i in range(32)]

row_indexer_handle.set(1)
column_indexer_handle.set(1)

#chip.read_all_block("ETROC2", "Pixel Config")
# Flip all bits
chip.read_all_block('ETROC2', 'Pixel Config')
for i in range(32):
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
    value = int(handle.get(),0)
    flipped = value ^ 0xff
    #print(f'Register {i} was {value}, flipped to {flipped}')
    handle.set(hex(flipped))
    flipped_values[i] = flipped

#print(f'Values should be {flipped_values}')

broadcast_handle.set(True)
chip.write_all_block("ETROC2", "Pixel Config")

for row in range(16):
    for col in range(16):
        row_indexer_handle.set(row)
        column_indexer_handle.set(col)

        chip.read_all_block('ETROC2', 'Pixel Config')

        for i in range(32):
            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

            value = int(handle.get(),0)
            if value != flipped_values[i]:
                print('col: ', col, 'row: ', row, f', Pixel readback values do not match the default values! Value: {value} should be {flipped_values[i]}')

In [ ]:
for row in range(16):
    for col in range(16):
        for i in range(32):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

            value = int(handle.get(),0)
            flipped = value ^ 0xff
            #print(f'Register {i} was {value}, flipped to {flipped}')

            handle.set(hex(flipped))

            chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)
            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped:
                print('col: ', col, 'row: ', row, ', Pixel byte readback values do not match the default values!')

## Read modify write per block (pixel)

In [ ]:


for row in range(16):
    for col in range(16):
        row_indexer_handle.set(row)
        column_indexer_handle.set(col)

        chip.read_all_block('ETROC2', 'Pixel Config')

        for i in range(32):
            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
            value = int(handle.get(),0)
            flipped = value ^ 0xff
            #print(f'Register {i} was {value}, flipped to {flipped}')
            handle.set(hex(flipped))
            flipped_values[i] = flipped

        chip.write_all_block('ETROC2', 'Pixel Config')

        for i in range(32):
            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

            value = int(handle.get(),0)
            if value != flipped_values[i]:
                print('col: ', col, 'row: ', row, f', Pixel readback values do not match the default values! Value: {value} should be {flipped_values[i]}')

## Fixed Xing test

In [ ]:
row_indexer_handle.set(0)
column_indexer_handle.set(0)
chip.read_all_block('ETROC2', 'Pixel Config')

for i in range(32):
    row_indexer_handle.set(0)
    column_indexer_handle.set(0)
    
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

    value = int(handle.get(),0)
    flipped = value ^ 0xff

    handle.set(hex(flipped))
    broadcast_handle.set(True)
    chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)

    failed = False
    for row in range(16):
        for col in range(16):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped:
                failed = True
                break
        if failed:
            break
    if failed:
        print(f"Broadcast failed for PixCfg{i}")

## Random order

In [ ]:
from numpy.random import permutation
row_indexer_handle.set(0)
column_indexer_handle.set(0)
chip.read_all_block('ETROC2', 'Pixel Config')

for i in permutation(32):
    row_indexer_handle.set(0)
    column_indexer_handle.set(0)
    
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

    value = int(handle.get(),0)
    flipped = value ^ 0xff

    handle.set(hex(flipped))
    broadcast_handle.set(True)
    chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)

    failed = False
    for row in range(16):
        for col in range(16):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped:
                failed = True
                break
        if failed:
            break
    if failed:
        print(f"Broadcast failed for PixCfg{i}")

## Flip all first, then check

In [5]:
from numpy.random import permutation
flipped_values = [None for i in range(32)]

b_row = 15  # This makes it always work
b_col = 15
b_row = 1 # Only works depending on read
b_col = 1
#b_row = 0 # Only works depending on read
#b_col = 0
row_indexer_handle.set(b_row)
column_indexer_handle.set(b_col)
chip.read_all_block('ETROC2', 'Pixel Config')

write_order = []
for i in permutation(32):
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
    write_order += [f"PixCfg{i}"]

    value = int(handle.get(),0)
    flipped = value ^ 0xff

    flipped_values[i] = flipped

    handle.set(hex(flipped))
    broadcast_handle.set(True)
    chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)  # So far, write check does not seem to make a difference
    #row_indexer_handle.set(0)  # Does not work for (0, 0)
    #column_indexer_handle.set(0)
    #row_indexer_handle.set(1)  # Does not work for (0, 0)
    #column_indexer_handle.set(1)
    #row_indexer_handle.set(2)  # Does not work for (0, 0)
    #column_indexer_handle.set(2)
    #row_indexer_handle.set(3)  # Does not work for (0, 0)
    #column_indexer_handle.set(3)
    #row_indexer_handle.set(4)  # Does not work for (0, 0)
    #column_indexer_handle.set(4)
    #row_indexer_handle.set(8)  # Does not work for (0, 0)
    #column_indexer_handle.set(8)
    #row_indexer_handle.set(0)  # Does not work for (0, 0)
    #column_indexer_handle.set(15)
    #row_indexer_handle.set(1)  # Does not work for (0, 0)
    #column_indexer_handle.set(15)
    #row_indexer_handle.set(8)  # Does not work for (0, 0)
    #column_indexer_handle.set(15)
    #row_indexer_handle.set(15)  # Does not work for (0, 0)
    #column_indexer_handle.set(0)
    #row_indexer_handle.set(15)  # Does not work for (0, 0)
    #column_indexer_handle.set(1)
    #row_indexer_handle.set(15)  # Does not work for (0, 0), (1, 1), works for (15, 15)
    #column_indexer_handle.set(8)
    #row_indexer_handle.set(14)  # Does not work for (1, 1), works for (15, 15)
    #column_indexer_handle.set(14)
    #row_indexer_handle.set(15)  # Works
    #column_indexer_handle.set(15)
    #chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
    row_indexer_handle.set(b_row)
    column_indexer_handle.set(b_col)
    #chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")

    #chip.read_all_block('ETROC2', 'Pixel Config')  # Does not work
    #sleep(1)  # No effect on breaking

print(f'Write order: {write_order}')
for i in range(32):
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

    failed = False
    row_fail = None
    col_fail = None
    for row in range(16):
        for col in range(16):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped_values[i]:
                failed = True
                row_fail = row
                col_fail = col
                print(f"Broadcast failed for PixCfg{i} on row {row_fail} col {col_fail} value is {value:#08b} should be {flipped_values[i]:#08b}")
    #            break
    #    if failed:
    #        break
    #if failed:
    #    print(f"Broadcast failed for PixCfg{i} on row {row_fail} col {col_fail} value is {value:#08b} should be {flipped_values[i]:#08b}")
    #    break

Write order: ['PixCfg2', 'PixCfg28', 'PixCfg25', 'PixCfg18', 'PixCfg8', 'PixCfg7', 'PixCfg6', 'PixCfg26', 'PixCfg10', 'PixCfg30', 'PixCfg15', 'PixCfg4', 'PixCfg19', 'PixCfg12', 'PixCfg13', 'PixCfg31', 'PixCfg17', 'PixCfg9', 'PixCfg21', 'PixCfg27', 'PixCfg0', 'PixCfg29', 'PixCfg5', 'PixCfg14', 'PixCfg1', 'PixCfg23', 'PixCfg20', 'PixCfg22', 'PixCfg3', 'PixCfg16', 'PixCfg24', 'PixCfg11']
Broadcast failed for PixCfg1 on row 0 col 0 value is 0b11111001 should be 0b000110
Broadcast failed for PixCfg1 on row 0 col 1 value is 0b11111001 should be 0b000110
Broadcast failed for PixCfg1 on row 0 col 2 value is 0b11111001 should be 0b000110
Broadcast failed for PixCfg1 on row 0 col 3 value is 0b11111001 should be 0b000110
Broadcast failed for PixCfg1 on row 0 col 4 value is 0b11111001 should be 0b000110
Broadcast failed for PixCfg1 on row 0 col 5 value is 0b11111001 should be 0b000110
Broadcast failed for PixCfg1 on row 0 col 6 value is 0b11111001 should be 0b000110
Broadcast failed for PixCfg1 on

In [ ]:
i2c_conn.Script_Helper._logger.setLevel(30)
for i in reg_list:
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

    row_fail = None
    col_fail = None
    for row in range(16):
        for col in range(16):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")
            value = int(handle.get(),0)

            if value != flipped_values[i]:
                row_fail = row
                col_fail = col
                print(f"Broadcast failed for PixCfg{i} on row {row_fail} col {col_fail} value is {value:#08b} should be {flipped_values[i]:#08b}")

# Single cell broadcast error demonstration

In [1]:
from notebooks.notebook_helpers import *

chip_address = 0x60
chip_name = "ET2_EPIR_Pair11"

port = "/dev/ttyACM0"

i2c_conn = i2c_connection(port,[chip_address],[None],[chip_name],[("1","1")])

i2c_conn.Script_Helper._logger.setLevel(30)

chip = i2c_conn.get_chip_i2c_connection(0x60)  ## Some I2C here? if yes, then just an ACK check

row_indexer_handle,_,_ = chip.get_indexer("row")
column_indexer_handle,_,_ = chip.get_indexer("column")
broadcast_handle,_,_ = chip.get_indexer("broadcast")

# reg_list = [2, 1]
reg_list = [9,10]
flipped_values = {}

b_row = 15  # This makes it always work
b_col = 15
# b_row = 1 # Only works depending on read
# b_col = 1
# b_row = 1 # Only works depending on read
# b_col = 15
# b_row = 1 # Only works depending on read
# b_col = 1
b_row = 0 # Only works depending on read
b_col = 0

row_indexer_handle.set(b_row)
column_indexer_handle.set(b_col)
chip.read_all_block('ETROC2', 'Pixel Config')  # Here multiple I2C block read commands

for i in reg_list:
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

    value = int(handle.get(),0)
    # flipped = value ^ 0xff
    flipped = 0x7e + i

    flipped_values[i] = flipped

    handle.set(hex(flipped))
    broadcast_handle.set(True)
    chip.write_register('ETROC2', 'Pixel Config', f"PixCfg{i}", write_check=False)  # So far, write check does not seem to make a difference

    #time.sleep(1)

for i in reg_list:
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
    print(f"first handle is {handle}")
    row_fail = None
    col_fail = None
    broadcast_handle.set(True)
    for row in range(15,0,-1):
        for col in range(15,0,-1):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)
            handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")      #read register
            value = int(handle.get(),0)
            print(f"read{value:#08x}")
            print(f"second handle is {handle}")
            # time.sleep(0.1)
            if value != flipped_values[i]:
                row_fail = row
                col_fail = col
                print(f"Broadcast failed for PixCfg{i} on row {row_fail} col {col_fail} value is {value} should be {flipped_values[i]:#08b} (on register read)")



# for row in range(16):
#     for col in range(16):
#         row_indexer_handle.set(row)
#         column_indexer_handle.set(col)

#         chip.read_all_block('ETROC2', 'Pixel Config')           #read all block

#         for i in reg_list:
#             handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")

#             value = int(handle.get(),0)
#             print(f"read2{value:#08x}")
#             if value != flipped_values[i]:
#                 row_fail = row
#                 col_fail = col
#                 print(f"Broadcast failed for PixCfg{i} on row {row_fail} col {col_fail} value is {value:#08b} should be {flipped_values[i]:#08b} (on block read)")

first handle is ETROC2_0_ETROC2_Reg32777
read0x000087
second handle is ETROC2_0_ETROC2_Reg40937
read0x000087
second handle is ETROC2_0_ETROC2_Reg40425
read0x000087
second handle is ETROC2_0_ETROC2_Reg39913
read0x000087
second handle is ETROC2_0_ETROC2_Reg39401
read0x000087
second handle is ETROC2_0_ETROC2_Reg38889
read0x000087
second handle is ETROC2_0_ETROC2_Reg38377
read0x000087
second handle is ETROC2_0_ETROC2_Reg37865
read0x000087
second handle is ETROC2_0_ETROC2_Reg37353
read0x000087
second handle is ETROC2_0_ETROC2_Reg36841
read0x000087
second handle is ETROC2_0_ETROC2_Reg36329
read0x000087
second handle is ETROC2_0_ETROC2_Reg35817
read0x000087
second handle is ETROC2_0_ETROC2_Reg35305
read0x000087
second handle is ETROC2_0_ETROC2_Reg34793
read0x000087
second handle is ETROC2_0_ETROC2_Reg34281
read0x000087
second handle is ETROC2_0_ETROC2_Reg33769
read0x000087
second handle is ETROC2_0_ETROC2_Reg40905
read0x000087
second handle is ETROC2_0_ETROC2_Reg40393
read0x000087
second hand

In [36]:
row_indexer_handle.set(2)
column_indexer_handle.set(1)

# handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{9}")

# value = int(handle.get(),0)

# print(f"read{value:#08x}")
# chip.read_all_block('ETROC2', 'Pixel Config')
chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{9}")      #read register

value2 = int(handle.get(),0)

print(f"Broadcast failed for PixCfg{i} on row {2} col {2} value is {value:#08b} should be {flipped_values[i]:#08b} (on register read)")

Broadcast failed for PixCfg9 on row 2 col 2 value is 0b011110 should be 0b011110 (on register read)


In [16]:
for i in reg_list:
    handle = chip.get_indexed_var('ETROC2', "Pixel Config", f"PixCfg{i}")
    print(handle)
    row_fail = None
    col_fail = None
    broadcast_handle.set(True)
    for row in range(5):
        for col in range(5):
            row_indexer_handle.set(row)
            column_indexer_handle.set(col)

            chip.read_register('ETROC2', 'Pixel Config', f"PixCfg{i}")      #read register
            value = int(handle.get(),0)
            print(f"read{value:#08x}")
            print(f"handle is {handle}")
            # time.sleep(0.1)
            if value != flipped_values[i]:
                row_fail = row
                col_fail = col
                print(f"Broadcast failed for PixCfg{i} on row {row_fail} col {col_fail} value is {value} should be {flipped_values[i]:#08b} (on register read)")


ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
handle is ETROC2_7_ETROC2_Reg34953
read0x000087
ha

## Low level

In [2]:
from usb_iss import UsbIss

# m - pixel matrix; s - status (if 0, config); b - broadcast
#                          m_s_b_col__row__reg
base_address           = 0b1_0_0_0010_0010_00000
broadcast_base_address = 0b1_0_1_0010_0010_00000
loop_base_address      = 0b1_0_0_0000_0000_00000

def swap_endian_16b(value):
    bit_low = value & 0xff
    bit_high = (value >> 8) & 0xff

    return (bit_low << 8) | bit_high

def broadcast_swap_bits_regs(iss: UsbIss, chip_address : int, reg_list: list[int], original_values: list[int]):
    flipped_values = {}
    for reg in reg_list:
        value = original_values[reg]
        flipped = value ^ 0xff
        flipped_values[reg] = flipped

        print(f"Flipping bits of PixCfg{reg} from {value:#010b} to {flipped:#010b}")
        iss.i2c.write_ad2(chip_address, swap_endian_16b(broadcast_base_address + reg), [flipped])
    print()

    return flipped_values

def individual_register_check(iss: UsbIss, chip_address : int, reg_list: list[int], flipped_values: dict[int, int]):
    print("Checking with individual register reads.")
    for reg in reg_list:
        print(f"Checking PixCfg{reg}:")
        for row in range(16):
            for col in range(16):
                value = iss.i2c.read_ad2(chip_address, swap_endian_16b(loop_base_address + reg + (row << 5) + (col << 9)), 1)[0]

                if value != flipped_values[reg]:
                    print(f"Broadcast failed for PixCfg{reg} on row {row} col {col} value is {value:#08b} should be {flipped_values[reg]:#08b} (on register read)")
    print()

def block_register_check(iss: UsbIss, chip_address : int, reg_list: list[int], flipped_values: dict[int, int], verbose = False):
    print("Checking with block register reads.")
    for row in range(16):
        for col in range(16):
            pix_regs = iss.i2c.read_ad2(chip_address, swap_endian_16b(loop_base_address + (row << 5) + (col << 9)), 32)

            for reg in range(len(pix_regs)):
                value = pix_regs[reg]
                if reg in reg_list:
                    if value != flipped_values[reg]:
                        print(f"Broadcast failed for PixCfg{reg} on row {row} col {col} value is {value:#08b} should be {flipped_values[reg]:#08b} (on block read)")
                    elif verbose:
                        print(f"Broadcast worked for PixCfg{reg} on row {row} col {col} value is {value:#08b}")
                elif verbose:
                    print(f'PixCfg{reg} on row {row} col {col} value is {value:#08b}')
    print()


port = "/dev/ttyACM0"
chip_address = 0x60

reg_list = [2, 1]
#reg_list = [9, 10]
#reg_list = list(range(32))

iss = UsbIss()

iss.open(port)
iss.setup_i2c(clock_khz=100, use_i2c_hardware=True)

iss.i2c.test(chip_address)

config_regs = iss.i2c.read_ad2(chip_address, swap_endian_16b(base_address), 32)
print("Original Config:")
for i in range(len(config_regs)):
    print(f'  - PixCfg{i}: {config_regs[i]:#010b}')
print()


flipped_values = broadcast_swap_bits_regs(iss, chip_address, reg_list, config_regs)

# Like this it fails in the "High level"
individual_register_check(iss, chip_address, reg_list, flipped_values)
block_register_check(iss, chip_address, reg_list, flipped_values)

# Like this it works in the "high level"
#block_register_check(iss, chip_address, reg_list, flipped_values)
#individual_register_check(iss, chip_address, reg_list, flipped_values)

Original Config:
  - PixCfg0: 0b01000000
  - PixCfg1: 0b00000110
  - PixCfg2: 0b00001111
  - PixCfg3: 0b00000101
  - PixCfg4: 0b10010101
  - PixCfg5: 0b00101001
  - PixCfg6: 0b01000010
  - PixCfg7: 0b00000111
  - PixCfg8: 0b10000001
  - PixCfg9: 0b11111010
  - PixCfg10: 0b11111111
  - PixCfg11: 0b11111111
  - PixCfg12: 0b00001111
  - PixCfg13: 0b00000000
  - PixCfg14: 0b00000000
  - PixCfg15: 0b11111111
  - PixCfg16: 0b11111111
  - PixCfg17: 0b11111111
  - PixCfg18: 0b11111111
  - PixCfg19: 0b00111111
  - PixCfg20: 0b00000000
  - PixCfg21: 0b00000000
  - PixCfg22: 0b11111100
  - PixCfg23: 0b11111111
  - PixCfg24: 0b00001111
  - PixCfg25: 0b00000000
  - PixCfg26: 0b00000000
  - PixCfg27: 0b00000000
  - PixCfg28: 0b00000000
  - PixCfg29: 0b00000000
  - PixCfg30: 0b00000000
  - PixCfg31: 0b00000000

Flipping bits of PixCfg2 from 0b00001111 to 0b11110000
Flipping bits of PixCfg1 from 0b00000110 to 0b11111001

Checking with individual register reads.
Checking PixCfg2:
Checking PixCfg1:

Che

In [ ]:
0b0_0_1_0000_0100_00010

0b0010000010000010
0b1000001000100000

0b1_0_0_0001_0001_00000

## Our tests

In [ ]:
register_under_test = "disDataReadout"
register_under_test = "upperTOT"
register_under_test = "lowerTOA"
register_under_test = "upperTOA"
register_under_test = "upperCal"

row_indexer_handle.set(15)
column_indexer_handle.set(15)
chip.read_all_block('ETROC2', 'Pixel Config')

# for row in range(16):
#     for col in range(16):
#         row_indexer_handle.set(row)
#         column_indexer_handle.set(col)
#         retrievevalue = i2c_conn.pixel_decoded_register_read(register_under_test, "Config", chip)
#         origvalue = int(retrievevalue, 16)
#         print(f'Pixel {row_indexer_handle.get()}, {column_indexer_handle.get()}')
#         print(bin(origvalue))
#         binaryorig = bin(origvalue)[2:]
#         bitflipvalue = origvalue ^ int("0xfffffffff", 16)
#         new_value = hex(int("0b" + (bin(bitflipvalue)[2:])[-len(binaryorig):], 0))
#         print(f"Trying to set to: {bin(int(new_value, 0))}")
#         broadcast_handle,_,_ = chip.get_indexer("broadcast")
#         broadcast_handle.set(True)
#         handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", register_under_test)
#         handle.set(new_value)
#         chip.write_decoded_value("ETROC2", "Pixel Config", register_under_test)

handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", register_under_test)

retrievevalue = handle.get()
origvalue = int(retrievevalue, 0)
print(f'Pixel {row_indexer_handle.get()}, {column_indexer_handle.get()}')
print(bin(origvalue))
binaryorig = bin(origvalue)[2:]
bitflipvalue = origvalue ^ int("0xfffffffff", 16)
print("bitflipvalue:",bin(bitflipvalue))
new_value = hex(int("0b" + (bin(bitflipvalue)[2:])[-len(binaryorig):], 0))
print(f"Trying to set to: {bin(int(new_value, 0))}")

In [ ]:
handle.set(new_value)
broadcast_handle.set(True)
chip.write_decoded_value("ETROC2", "Pixel Config", register_under_test,write_check=False)

In [ ]:
for row in range(16):
    for col in range(16):
        column_indexer_handle.set(col)
        row_indexer_handle.set(row)

        chip.read_decoded_value("ETROC2", "Pixel Config", register_under_test)
        handle = chip.get_decoded_indexed_var("ETROC2", "Pixel Config", register_under_test)
        value = handle.get()

        if int(value,0) != int(new_value,0):
            print(f'Error on pixel ({row},{col}): {value} should be {new_value}')